In [1]:
# Author: Ron Weiss <ronweiss@gmail.com>, Gael Varoquaux
# License: BSD 3 clause

# $Id$

import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import time

from sklearn import datasets
from sklearn.cross_validation import StratifiedKFold
from sklearn.cluster import KMeans
from sklearn.externals.six.moves import xrange
from sklearn.mixture import GMM
from sklearn import metrics


%matplotlib inline 

In [29]:
#replaced with online news popularity dataset
original = pd.read_csv('A3_OnlineNewPopularity_sharegrps_labeled.csv')
us_file = original.drop(labels=[' timedelta','url', 'shares_2', 'shares_3', 'shares_5', 'shares_log'], axis=1)
#target = np.array(original['shares_2'])
news_data = np.array(us_file)
X = news_data
#us_file.isnull().sum()

### SET THE LABELS FROM CLUSTERING
n_clusters = 4

##############################################################################
# Compute clustering with Means

k_means = KMeans(init='k-means++', n_clusters=n_clusters, n_init=10)
t0 = time.time()
k_means.fit(X)
t_batch = time.time() - t0
k_means_labels = k_means.labels_
k_means_cluster_centers = k_means.cluster_centers_
k_means_labels_unique = np.unique(k_means_labels)

y= k_means_labels
target = y
print k_means_labels

[1 1 1 ..., 0 0 2]


In [30]:
# Break up the dataset into non-overlapping training (75%) and testing
# (25%) sets.
skf = StratifiedKFold(target, n_folds=4)
# Only take the first fold.
train_index, test_index = next(iter(skf))


X_train = news_data[train_index]
y_train = target[train_index]
X_test = news_data[test_index]
y_test = target[test_index]
n_classes = len(np.unique(y_train))

print n_classes

4


In [31]:
#Compare GMM outputs content with Kmeans
output_class =8
g = GMM(n_components=output_class,covariance_type='spherical') #add diagonal?
g.fit(news_data)
g_label_pred = g.predict(X)

In [32]:
from collections import Counter
gmm_freqs = Counter(g_label_pred)
print gmm_freqs

Counter({6: 7175, 0: 6819, 4: 6773, 7: 5710, 2: 5054, 3: 4804, 1: 3088, 5: 221})


In [33]:
#output for part #5 of the homework
print g_label_pred
print pd.unique(g_label_pred)
us_file['Target_GMM8'] = g_label_pred
us_file.to_csv('News_GMM1_k8.csv', index=False)
#us_file.columns

[1 1 1 ..., 6 0 7]
[1 7 3 5 6 0 4 2]


In [48]:
us_gmm_groupby = us_file.groupby('Target_GMM8').mean()
us_gmm_groupby


,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,num_videos,average_token_length,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
Target_GMM8,,,,,,,,,,,,,,,,,,,,,
0,10.659481,616.854524,0.516398,0.975216,0.666763,10.801437,3.078164,4.723127,0.866696,4.582696,...,0.093267,0.761153,-0.261078,-0.553498,-0.101069,0.274402,0.062950,0.337218,0.149091,2356.741751
1,9.815415,476.823187,0.562372,0.991256,0.705744,9.191386,3.226360,3.443653,1.354275,4.620080,...,0.100233,0.770988,-0.253161,-0.481075,-0.114249,0.283980,0.098237,0.344508,0.165347,2795.454987
2,10.680253,492.457855,0.532695,0.950732,0.665062,10.789869,3.163237,4.677879,1.919865,4.456448,...,0.098018,0.736239,-0.266258,-0.515141,-0.112611,0.302156,0.071189,0.342407,0.166158,3410.662643
3,9.697336,507.300791,0.552194,0.991674,0.696223,10.259575,3.437552,3.770400,0.609284,4.623711,...,0.096317,0.760441,-0.247079,-0.479539,-0.109834,0.255876,0.074266,0.351404,0.138015,2940.644255
4,10.395541,649.644323,0.513805,0.983464,0.667186,11.854422,3.846301,4.861804,0.713864,4.614767,...,0.090042,0.780182,-0.250904,-0.535198,-0.097207,0.266592,0.071402,0.342501,0.148378,2589.643880
5,10.547511,425.728507,0.547637,0.936652,0.682766,7.443439,2.257919,2.266968,2.692308,4.357750,...,0.099074,0.707191,-0.249687,-0.481549,-0.112882,0.325397,0.090724,0.325312,0.184474,3162.104072
6,10.634286,564.123763,0.627142,1.126690,0.761829,11.684042,3.249059,5.256028,0.987038,4.622647,...,0.097983,0.764879,-0.267368,-0.539305,-0.109718,0.282289,0.065908,0.340840,0.153488,2935.232195
7,10.445884,441.260771,0.530610,0.923292,0.657432,10.481261,3.022242,4.274956,2.507180,4.320950,...,0.095529,0.722596,-0.266369,-0.512086,-0.114258,0.312827,0.071082,0.338500,0.176874,6872.214886


In [ ]:
us_gmm_groupby.to_csv('gmm_groupby_output.csv')

In [34]:
# Try GMMs using different types of covariances.
classifiers = dict((covar_type, GMM(n_components=n_classes,
                    covariance_type=covar_type))
                   for covar_type in ['spherical', 'diag'])
                   #for covar_type in ['spherical', 'diag', 'tied', 'full'])
                                                            
#n_classifiers = len(classifiers)
class_range = [3,4,5,6,7,8,9,10]

#add a changing kemans label too
for index, (name, classifier) in enumerate(classifiers.items()):
    for n_classes in class_range:
        # Since we have class labels for the training data, we can
        # initialize the GMM parameters in a supervised manner.
        classifier.means_ = np.array([X_train[y_train == i].mean(axis=0)
                                      for i in xrange(n_classes)])

        # Train the other parameters using the EM algorithm.
        classifier.fit(X_train)
        y_train_pred = classifier.predict(X_train)
        train_accuracy = np.mean(y_train_pred.ravel() == y_train.ravel()) * 100
        print '%f %s Train accuracy: %.1f, converged: %s' % (n_classes, name, train_accuracy, classifier.converged_)

        y_test_pred = classifier.predict(X_test)
        test_accuracy = np.mean(y_test_pred.ravel() == y_test.ravel()) * 100
        print '%f %s Test accuracy: %.1f' % (n_classes, name, test_accuracy)



3.000000 spherical Train accuracy: 67.2, converged: True
3.000000 spherical Test accuracy: 70.1
4.000000 spherical Train accuracy: 10.2, converged: True
4.000000 spherical Test accuracy: 7.3
5.000000 spherical Train accuracy: 45.7, converged: True
5.000000 spherical Test accuracy: 48.3
6.000000 spherical Train accuracy: 10.2, converged: True
6.000000 spherical Test accuracy: 7.3
7.000000 spherical Train accuracy: 67.2, converged: True
7.000000 spherical Test accuracy: 70.1
8.000000 spherical Train accuracy: 21.9, converged: True
8.000000 spherical Test accuracy: 21.9
9.000000 spherical Train accuracy: 2.5, converged: True
9.000000 spherical Test accuracy: 2.0
10.000000 spherical Train accuracy: 21.9, converged: True
10.000000 spherical Test accuracy: 21.9
3.000000 diag Train accuracy: 36.7, converged: False
3.000000 diag Test accuracy: 22.8
4.000000 diag Train accuracy: 38.9, converged: False
4.000000 diag Test accuracy: 26.9
5.000000 diag Train accuracy: 13.6, converged: False
5.00000

RuntimeError: EM algorithm was never able to compute a valid likelihood given initial parameters. Try different init parameters (or increasing n_init) or check for degenerate data.

In [31]:
# GMM labels compared with Kmeans labeling
# Get values to plot and compare

np.random.seed(123)
cluster_range = [2,3,4,5,6,7,8,9,10]
sample_size = 500

for n_clusters in cluster_range: 

    k_means = KMeans(init='k-means++', n_clusters=n_clusters, n_init=10)
    t0 = time.time()
    k_means.fit(news_data)
    t_batch = time.time() - t0
    k_means_labels = k_means.labels_
    k_means_cluster_centers = k_means.cluster_centers_

    g = GMM(n_components=n_clusters,covariance_type='spherical')
    g_t0 = time.time()
    g.fit(news_data)
    g_t_batch = time.time() - g_t0
    g_label_pred = g.predict(X)
    
    
    print('%.3f, %.2fs,  %i,   %.3f,   %.3f,   %.3f,   %.3f,   %.3f,    %.3f,   %.3f,   %.3f,   %.3f,'
      % (n_clusters,(time.time() - t0), k_means.inertia_,
         metrics.homogeneity_score(k_means_labels, g_label_pred),
         metrics.completeness_score(k_means_labels, g_label_pred),
         metrics.v_measure_score(k_means_labels, g_label_pred),
         metrics.adjusted_rand_score(k_means_labels, g_label_pred),
         metrics.adjusted_mutual_info_score(k_means_labels,  g_label_pred),
         metrics.silhouette_score(news_data, g_label_pred,
                                  metric='euclidean',
                                  sample_size=sample_size),
         metrics.silhouette_score(news_data, g_label_pred,
                                  metric='manhattan',
                                  sample_size=sample_size),
         metrics.silhouette_score(news_data, g_label_pred,
                                  metric='cityblock',
                                  sample_size=sample_size),
         metrics.silhouette_score(news_data, g_label_pred,
                                  metric='cosine',
                                  sample_size=sample_size)
        )
      )


2.000, 0.96s,  959273160582714,   0.942,   0.961,   0.951,   0.981,   0.942,    0.778,   0.776,   0.770,   0.683,
3.000, 4.41s,  596446571502952,   0.576,   0.526,   0.550,   0.372,   0.526,    0.313,   0.258,   0.229,   0.208,
4.000, 4.22s,  436139861874352,   0.723,   0.697,   0.710,   0.621,   0.697,    0.453,   0.424,   0.448,   0.152,
5.000, 4.86s,  315231323361863,   0.788,   0.615,   0.691,   0.536,   0.615,    0.418,   0.348,   0.368,   0.156,
6.000, 7.55s,  249076372081758,   0.757,   0.646,   0.697,   0.562,   0.646,    0.406,   0.310,   0.298,   0.116,
7.000, 8.44s,  184871217478282,   0.788,   0.669,   0.724,   0.579,   0.669,    0.410,   0.320,   0.324,   0.134,
8.000, 8.48s,  157837088633236,   0.796,   0.699,   0.745,   0.606,   0.699,    0.362,   0.311,   0.319,   0.182,
9.000, 10.05s,  134573470393785,   0.812,   0.717,   0.761,   0.618,   0.716,    0.387,   0.309,   0.309,   0.176,
10.000, 11.75s,  122591625882848,   0.791,   0.663,   0.722,   0.577,   0.663,    0.428

In [ ]:
print(__doc__)

import itertools

import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl

from sklearn import mixture

# Number of samples per component
n_samples = 500

# Generate random sample, two components
# np.random.seed(0)
# C = np.array([[0., -0.1], [1.7, .4]])
# X = np.r_[np.dot(np.random.randn(n_samples, 2), C),
#           .7 * np.random.randn(n_samples, 2) + np.array([-6, 3])]

lowest_bic = np.infty
bic = []
n_components_range = range(1, 60)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a mixture of Gaussians with EM
        gmm = mixture.GMM(n_components=n_components, covariance_type=cv_type)
        gmm.fit(X)
        bic.append(gmm.bic(X))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm

bic = np.array(bic)
color_iter = itertools.cycle(['k', 'r', 'g', 'b', 'c', 'm', 'y'])
clf = best_gmm
bars = []

# Plot the BIC scores
spl = plt.subplot(2, 1, 1)
for i, (cv_type, color) in enumerate(zip(cv_types, color_iter)):
    xpos = np.array(n_components_range) + .2 * (i - 2)
    bars.append(plt.bar(xpos, bic[i * len(n_components_range):
                                  (i + 1) * len(n_components_range)],
                        width=.2, color=color))
plt.xticks(n_components_range)
plt.ylim([bic.min() * 1.01 - .01 * bic.max(), bic.max()])
plt.title('BIC score per model')
xpos = np.mod(bic.argmin(), len(n_components_range)) + .65 +\
    .2 * np.floor(bic.argmin() / len(n_components_range))
plt.text(xpos, bic.min() * 0.97 + .03 * bic.max(), '*', fontsize=14)
spl.set_xlabel('Number of components')
spl.legend([b[0] for b in bars], cv_types)

# Plot the winner
splot = plt.subplot(2, 1, 2)
Y_ = clf.predict(X)
for i, (mean, covar, color) in enumerate(zip(clf.means_, clf.covars_,
                                             color_iter)):
    v, w = linalg.eigh(covar)
    if not np.any(Y_ == i):
        continue
    plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], .8, color=color)

    # Plot an ellipse to show the Gaussian component
    angle = np.arctan2(w[0][1], w[0][0])
    angle = 180 * angle / np.pi  # convert to degrees
    v *= 4
    ell = mpl.patches.Ellipse(mean, v[0], v[1], 180 + angle, color=color)
    ell.set_clip_box(splot.bbox)
    ell.set_alpha(.5)
    splot.add_artist(ell)

plt.xlim(-10, 10)
plt.ylim(-3, 6)
plt.xticks(())
plt.yticks(())
plt.title('Selected GMM: full model, 2 components')
plt.subplots_adjust(hspace=.35, bottom=.02)
plt.show()

In [ ]:
color_iter = itertools.cycle(['r', 'g', 'b', 'c', 'm'])


for i, (clf, title) in enumerate([
        (mixture.GMM(n_components=10, covariance_type='full', n_iter=100),
         "Expectation-maximization")]):

    clf.fit(X)
    splot = plt.subplot(3, 1, 1 + i)
    Y_ = clf.predict(X)
    for i, (mean, covar, color) in enumerate(zip(
            clf.means_, clf._get_covars(), color_iter)):
        v, w = linalg.eigh(covar)
        u = w[0] / linalg.norm(w[0])
        # as the DP will not use every component it has access to
        # unless it needs it, we shouldn't plot the redundant
        # components.
        if not np.any(Y_ == i):
            continue
        plt.scatter(X[Y_ == i, 0], X[Y_ == i, 1], .8, color=color)

        # Plot an ellipse to show the Gaussian component
        angle = np.arctan(u[1] / u[0])
        angle = 180 * angle / np.pi  # convert to degrees
        ell = mpl.patches.Ellipse(mean, v[0], v[1], 180 + angle, color=color)
        ell.set_clip_box(splot.bbox)
        ell.set_alpha(0.5)
        splot.add_artist(ell)

    plt.xlim(-6, 4 * np.pi - 6)
    plt.ylim(-5, 5)
    plt.title(title)
    plt.xticks(())
    plt.yticks(())

plt.show()